In [1]:
import sys
import logging
import time
import random
import numpy as np
from collections import defaultdict, namedtuple
from optparse import OptionParser

# Required libraries
import h5py
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization as BN

import IPython.display
import matplotlib.pylab as plt
%matplotlib inline

import sys
sys.path.append('../')
# d-script imports
from data_iters.minibatcher import MiniBatcher
from data_iters.iam_hdf5_iterator import IAM_MiniBatcher
from fielutil import fielnet
import viz_tools.array_to_png as a2p

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled)


## Parameter settings

In [2]:
hdf5_file = '/fileserver/icdar13/benchmarking-processed/author_icdar_be.hdf5'
neural_network_params = '../convnets/fielnet/fielnet.hdf5'
num_authors=100
num_forms_per_author=-1
shingle_dim=(120,120)
use_form=True

batch_size=32
lr = 0.01

## Initialize Classification Minibatcher

In [3]:
from class_icdar_iterator import StepShingler

## Load a neural network model

In [5]:
model=fielnet(neural_network_params, layer='fc7')
print "Compiling model"
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.7, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
print "Finished compilation"

Compiling model
Finished compilation


## Processing to get the image hash
1. Get the shingle through the step shingler
2. Extract the feature with FielNet
3. Aggregate all the features with mean averaging

In [ ]:
def preprocshingle( shingle ):
    
    return shingle

In [17]:
icdar_hdf5 = h5py.File(hdf5_file, "r")
authors = icdar_hdf5.keys()

k = 0; lb=0; le=0
for author in authors:
    fragments = icdar_hdf5[author].keys()
    for fragment in fragments:
        print "Working on author %s on image %s" %(author, fragment)
        img = icdar_hdf5[author][fragment]
        imgshingler = StepShingler(img, hstep=90, vstep=120, shingle_size=(120,120))
        for imgshingle in imgshingler:
            feature_batch = model.predict( img_minibatch )
        average_feature = feature_batches.mean(axis=1)
        

        # shinglebuffer = np.zeros(1)
        # for icdar_shingle, author_id, fragment_id in icdar_shingles:
        #     shinglebuffer[i] = imgshingler()
imgshingle.shape

Working on author 001 on image 001_1.tif
Got imgshingle


(120, 120)

In [27]:
ibuflen = 3200
vskip=120
hskip=90
input_buffer = np.zeros((ibuflen, 1, shingle_dim[0], shingle_dim[1]))
output_buffer = np.zeros((0, 128))

icdar_hdf5 = h5py.File(hdf5_file, "r")
authors = icdar_hdf5.keys()
for author in authors:
    fragments = icdar_hdf5[author].keys()
    for fragment in fragments:
        print "Working on author %s on image %s" %(author, fragment)
        im = icdar_hdf5[author][fragment]

        k = 0; lb=0; le=0
        for i in xrange(0, im.shape[0] - shingle_dim[0] + vskip, vskip):
            for j in xrange(0, im.shape[1] - shingle_dim[1] + hskip, hskip):
                im_shingle = im[i:(i+shingle_dim[0]), j:(j+shingle_dim[1])]
                if im_shingle.shape != (120,120):
                    print "Reached the end of shingle %d, %d and divided unevenly" %(i,j)
                    break
                input_buffer[k] = np.expand_dims( im[i:(i+shingle_dim[0]), j:(j+shingle_dim[1])], 0 )
                k+=1; le+=1;
                # Dump buffer and do inference
                if k==ibuflen:
                    np.concatenate( (output_buffer, model.predict( input_buffer )) )
                    pctdone = float(le)/obuflen*100
                    sys.stdout.write("\033[F") # Cursor up one line
                    print ("Finished output_buffer fill (%d - %d) / %d : %f %%" %( lb, le, obuflen, float(le)/obuflen*100 ))
                    k=0; lb=le
        break
        
        
        # shinglebuffer = np.zeros(1)
        # for icdar_shingle, author_id, fragment_id in icdar_shingles:
        #     shinglebuffer[i] = imgshingler()
im_shingle.shape

Working on author 001 on image 001_1.tif
Reached the end of shingle 0, 2340 and divided unevenly
Reached the end of shingle 120, 2340 and divided unevenly
Reached the end of shingle 240, 2340 and divided unevenly
Reached the end of shingle 360, 0 and divided unevenly
Working on author 002 on image 002_1.tif
Reached the end of shingle 480, 0 and divided unevenly
Working on author 003 on image 003_1.tif
Reached the end of shingle 480, 0 and divided unevenly
Working on author 004 on image 004_1.tif
Reached the end of shingle 360, 0 and divided unevenly
Working on author 005 on image 005_1.tif
Reached the end of shingle 360, 0 and divided unevenly
Working on author 006 on image 006_1.tif
Reached the end of shingle 360, 0 and divided unevenly
Working on author 007 on image 007_1.tif
Reached the end of shingle 360, 0 and divided unevenly
Working on author 008 on image 008_1.tif
Reached the end of shingle 240, 0 and divided unevenly
Working on author 009 on image 009_1.tif
Reached the end of 

(16, 120)

## Feature aggregator

## Save Features for Classifier? (Into Train/Test/Validation?)